In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision.transforms as transforms
import torchvision.datasets as vdatasets
import torchvision.utils as vutils
import random
import os, pickle
from tensorboardX import SummaryWriter
torch.manual_seed(1)

DATA_PATH = os.environ['DATA_PATH']
USE_CUDA = torch.cuda.is_available()

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# 텐서보드 포트 설정
port = pickle.load(open("port.info","rb"))[os.getcwd().split("/")[-2]]

# 텐서보드 데이터 파일 초기화
try:
    shutil.rmtree('runs/')
except:
    pass

https://arxiv.org/pdf/1512.03385.pdf

## Residual Module 

In [3]:
class Residual(nn.Module):
    def __init__(self,in_channels,out_channels=None,reduce_grid=False):
        super(Residual,self).__init__()
        
        if out_channels:
            self.increase_depth = True
        else:
            self.increase_depth = False
            out_channels = in_channels
            
        if reduce_grid:
            first_stride = 2
        else:
            first_stride = 1
        
        self.residual_layer = nn.Sequential(
                                                    nn.Conv2d(in_channels,out_channels,3,first_stride,1),
                                                    nn.BatchNorm2d(out_channels),
                                                    nn.ReLU(),
                                                    nn.Conv2d(out_channels,out_channels,3,1,1),
                                                    nn.BatchNorm2d(out_channels),
                                                    nn.ReLU(),
                                                )
        if self.increase_depth:
            self.conv1_layer = nn.Sequential(
                                                            nn.Conv2d(in_channels,out_channels,1,first_stride),
                                                            nn.BatchNorm2d(out_channels),
                                                            nn.ReLU(),
                                                            )
            
    def forward(self,inputs):
        residual = self.residual_layer(inputs) # F(x)
        if self.increase_depth:
            inputs = self.conv1_layer(inputs)
        return F.relu(residual+inputs) # F.relu(F(x)+x)

In [5]:
writer = SummaryWriter(comment='-basic-residual2')
sample_images = torch.randn(2,3,224,224)
residual = Residual(3)
# output = residual(Variable(sample_images))
writer.add_graph(residual,(Variable(sample_images),))
writer.close()

In [5]:
!tensorboard --logdir runs --port 6006

TensorBoard 0.4.0rc3 at http://dsksd-tf:6006 (Press CTRL+C to quit)
^C


## ResNet-34

In [11]:
class ResNet34(nn.Module):
    
    def __init__(self,base_channels,output_size):
        super(ResNet34,self).__init__()
        
        self.layer1 = nn.Sequential(
                                    nn.Conv2d(3,base_channels,7,2,3),
                                    nn.BatchNorm2d(base_channels),
                                    nn.ReLU(),
                                    nn.MaxPool2d(3,2,1)
                                    )
        
        self.residual_layers = nn.Sequential(
                                                Residual(base_channels),
                                                Residual(base_channels),
                                                Residual(base_channels),
                                                Residual(base_channels,base_channels*2,True), # increase depth, reduce grid
                                                Residual(base_channels*2),
                                                Residual(base_channels*2),
                                                Residual(base_channels*2),
                                                Residual(base_channels*2,base_channels*4,True),
                                                Residual(base_channels*4),
                                                Residual(base_channels*4),
                                                Residual(base_channels*4),
                                                Residual(base_channels*4),
                                                Residual(base_channels*4),
                                                Residual(base_channels*4,base_channels*8,True),
                                                Residual(base_channels*8),
                                                Residual(base_channels*8),
                                                )
        self.global_avg = nn.AvgPool2d(7,7) # base_channels*8,1,1
        self.fc = nn.Linear(base_channels*8,output_size)
        
    def forward(self,inputs):
        outputs = self.layer1(inputs)
        outputs = self.residual_layers(outputs)
        outputs = self.global_avg(outputs)
        
        # B,512,1,1 -> B,512 => B,512*1*1
        #outputs.view(B,-1)
        
        outputs = self.fc(outputs.view(outputs.size(0),-1)) # B,1000
        
        return outputs

## Sanity Check 

In [12]:
BATCH_SIZE = 2
model = ResNet34(64,10)
writer = SummaryWriter(comment="-resnet")

In [13]:
sample_x = torch.randn(BATCH_SIZE,3,224,224)
# sample_y = torch.randperm(BATCH_SIZE)

In [16]:
%time
model.train()
preds = model(Variable(sample_x))
writer.add_graph(model,(Variable(sample_x),))
writer.close()

CPU times: user 6 µs, sys: 2 µs, total: 8 µs
Wall time: 14.3 µs


In [17]:
!tensorboard --logdir runs --port 6006

TensorBoard 0.4.0rc3 at http://dsksd-tf:6006 (Press CTRL+C to quit)
^C
